# Joining Tables in SQL #
## Prerequisites ##
- For this tutorial you should already have completed the 1_intro_to_sql tutorial. Your schools.db file should contain the students table with multiple students in it, with a sequence that automatically generates student ids. 

## Primary and Foreign Keys ##

Primary keys are unique identifiers for each row in a table. They are used to ensure that each row in a table is unique. Foreign keys are used to link two tables together. They are used to ensure that the value in a column in one table is present in another table. 

We have already created a `students` table. For our next move we will create an `awards` table. This table will contain information about awards that students have won. Each award can only be given to one student, so we will need to link the `awards` table to the `students` table. This saves us from duplicating the relevant information about a student in the `awards` table.

We will create the `awards` table with the following columns:
- `award_id` - a unique identifier for each award
- `student_id` - the id of the student who won the award. This is a foreign key that links to the `students` table
- `award_name` - the name of the award
- `award_date` - the date the award was won (defaulting to the current date)


In [1]:
# Importing relevant libraries. This needs to run every time the script is run.
import duckdb

In [ ]:

# Create the awards table in the database (run this script only once)

with duckdb.connect('../data/school.db') as con:
    con.sql('''
            CREATE TABLE awards (
            award_id INTEGER PRIMARY KEY,
            award TEXT,
            year INTEGER,
            student_id INTEGER REFERENCES students(student_id)
        );
    ''')